# Search the articles

This notebook will search the articles for the project exploiting the [Media Cloud](https://mediacloud.org/) database.
One have to set the customizable parameters:

1. `PATH_TO_DATA` is the Path object for saving the constructed DataFrames;
2. `MY_KEY` parameter is the user key every Media Cloud user has been provided once signed up, for more info go [here](https://github.com/mediacloud/backend/blob/master/doc/api_2_0_spec/api_2_0_spec.md#authentication);
3. `MIN_LEGTH` parameter is the minimiun length that a word must have to be considered for the word vector;
4. `MIN_FREQUENCY` parameter is the minimum frequency a word must appear in at least one article to be considered for the word vector;
5. `MAX_ARTICLES` parameter set the maximum number of articles to search.

In [116]:
import requests
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
from joblib import Parallel, delayed

# Constants, NOT TO BE CHANGED
MEDIA_CLOUD_URL = 'https://api.mediacloud.org'
STORIES_LIST = '/api/v2/stories_public/list/'
STORIES_WORD_MATRIX = '/api/v2/stories_public/word_matrix/'
# These following have to be customized
PATH_TO_DATA = Path('../data')
MY_KEY = '66aa9cf8dbd642b0e47f6811764cbe451a84d9429b8d2b3647c97c0af8fd40f5'
MIN_LENGTH = 10
MIN_FREQUENCY = 10
MAX_ARTICLES = 100

The following cell will run two queries.
The first will get the articles ID, while the second will get the word matrix (used in the project).
One has to customize the filters for getting what he/she is interested in.
Remind that the epidemic model works fine for events that have a rapid spreading (popular, interesting) and then die, because of that one has to pay attention not only to the argument but also to the time windows.
For more info about constructing the query look [here](https://github.com/mediacloud/backend/blob/master/doc/api_2_0_spec/api_2_0_spec.md#query-parameters-5).
For 3k articles this should take ~1.5min.

In [117]:
# Initialize articles' ids list
stories_id = []
# Filter to pass to the query
params = {
    # numer of articles per query
    'rows': MAX_ARTICLES,
    # filtering the language of the article to have consistent words
    'q': 'language:en',
    # main filters: arguments and so on
    'fq': 'text:Moderna AND text:vaccine AND text:covid',
    # personal key to pass to the query
    'key': MY_KEY,
}
# Get the articles id
r = requests.get(
    MEDIA_CLOUD_URL+STORIES_LIST,
    params=params,
    headers={'Accept': 'application/json'})
# Fill articles' id list
stories_info = r.json()
# Get word matrix of the query
r = requests.get(
    MEDIA_CLOUD_URL+STORIES_WORD_MATRIX,
    params=params,
    headers={'Accept': 'application/json'})
# Retrieve the results to be analyzed
stories_word_info = r.json()

  0%|          | 4/2721 [01:08<12:56:18, 17.14s/columns]


The following cell will build the words vector DataFrame using `pandas` library.
The DataFrame will have a column (`id`) with the id of the article, and also one column per word named with that word.
For 3k articles this should take TODO.

In [ ]:
words_df = pd.DataFrame()
n_words = len(stories_word_info['word_list'])
columns = [stories_word_info['word_list'][i][0] for i in range(len(stories_word_info['word_list']))]
columns.insert(0,'id')
keys_iterator = tqdm(
    stories_word_info['word_matrix'].keys(),
    leave=True,
    unit='columns',
)
def process(key):
    word_vector_dict = stories_word_info['word_matrix'][key]
    word_vector = np.zeros(n_words)
    for kkey in word_vector_dict.keys():
        word_vector[eval(kkey)] = word_vector_dict[kkey]
    word_vector = list([key])+list(word_vector)
    return pd.DataFrame([word_vector], columns=columns)
    # words_df = words_df.append(pd.DataFrame([word_vector], columns=columns))
    
results = Parallel(n_jobs=10)(delayed(process)(i) for i in keys_iterator)
results = pd.concat(results, axis=0)
results

The following cell will drop out the column corresponding to the word that should not be counted for the words vectors as set by the parameters `MIN_LENGTH` and `MIN_FREQUENCY`.
For 3k articles this should take TODO.

In [ ]:
cols_iterator = tqdm(
    words_df.columns,
    leave=True,
    unit='columns',
)
for col in cols_iterator:
    if len(col) < MIN_LENGTH:
        words_df = words_df.drop(col,1)
        continue
    words_df[col] = pd.to_numeric(words_df[col])
    if len(words_df[words_df[col]>MIN_FREQUENCY]) == 0:
        words_df = words_df.drop(col,1)
words_df

""
0
0
0
0
0
0
0
0
0
0


The following cell will build the article info DataFrame using `pandas` library.
The DataFrame will have a column (`id`) with the id of the article.
Column `timestamp` will carry the data in which the article has been published.
Columns `source` will carry the media id of the media in which the article has been published.

In [ ]:
info_df = pd.DataFrame()
info_df['id'] = [stories_info[i]['stories_id'] for i in range(len(stories_info))]
info_df['timestamp'] = [stories_info[i]['publish_date'] for i in range(len(stories_info))]
info_df['timestamp'] = info_df['timestamp'].map(lambda x : x[:10])
info_df['timestamp'] = pd.to_datetime(info_df.timestamp)
info_df['source'] = [stories_info[i]['media_id'] for i in range(len(stories_info))]
info_df

Finally, the DataFrames are save in the data folder, defined by `PATH_TO_DATA`.

In [ ]:
words_df.to_csv(PATH_TO_DATA/'words_dataframe.csv')
info_df.to_csv(PATH_TO_DATA/'info_dataframe.csv')